<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="4em" style="font-weight:bold;color:#3f8dbf;">탐색적 데이터 분석</font><br>

### 전체 판매 프로세스
1. 해당 쇼핑몰에 중소업체가 계약을 맺고
2. 중소업체가 해당 쇼핑몰에 직접 상품을 올리고
2. 고객이 구매하면, 중소업체가 Olist가 제공하는 물류 파트너를 활용해서 배송을 하고,
3. 고객이 상품을 받으면, 고객에게 이메일 survey 가 전송되고,
4. 고객이 이메일 survey 에 별점과 커멘트를 남겨서 제출하게 됨
    
### 데이터 출처
- 브라질에서 가장 큰 백화점의 이커머스 쇼핑몰 (https://olist.com/)
  - 2016년도부터 2018년도 9월까지의 100k 개의 구매 데이터 정보
  - 구매 상태, 가격, 지불수단, 물류 관련, 리뷰관련, 상품 정보, 구매자 지역 관련 정보


</div>

#### 기존의 사전 작업

In [41]:
import pandas as pd
path = './olist_ecommerce_data/'
products = pd.read_csv( path + 'olist_products_dataset.csv', encoding = 'utf-8-sig')
customers = pd.read_csv( path + 'olist_customers_dataset.csv', encoding = 'utf-8-sig')
geolocation = pd.read_csv( path + 'olist_geolocation_dataset.csv', encoding = 'utf-8-sig')
order_items = pd.read_csv( path + 'olist_order_items_dataset.csv', encoding = 'utf-8-sig')
payments = pd.read_csv( path + 'olist_order_payments_dataset.csv', encoding = 'utf-8-sig')
reviews = pd.read_csv( path + 'olist_order_reviews_dataset.csv', encoding = 'utf-8-sig')
orders = pd.read_csv( path + 'olist_orders_dataset.csv', encoding = 'utf-8-sig')
sellers = pd.read_csv( path + 'olist_sellers_dataset.csv', encoding = 'utf-8-sig')
category_name = pd.read_csv( path + 'product_category_name_translation.csv', encoding = 'utf-8-sig')

#### orders의 null 인 데이터 삭제하고, payments와 합치기

In [42]:
orders = orders.dropna()
merged_order = pd.merge(orders, payments, on = 'order_id')
merged_order.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'payment_sequential', 'payment_type', 'payment_installments',
       'payment_value'],
      dtype='object')

In [43]:
merged_order_payment_date = merged_order[['order_purchase_timestamp', 'payment_value']].copy()

### [ 1. 거래 건수 확인하기 ]

###  (1) 월별

In [44]:
merged_order_payment_date

,order_purchase_timestamp,payment_value
0,2017-10-02 10:56:33,18.12
1,2017-10-02 10:56:33,2.00
2,2017-10-02 10:56:33,18.59
3,2018-07-24 20:41:37,141.46
4,2018-08-08 08:38:49,179.12
...,...,...
100734,2017-03-09 09:54:05,85.08
100735,2018-02-06 12:58:58,195.00
100736,2017-08-27 14:46:43,271.01
100737,2018-01-08 21:28:27,441.16


In [45]:
merged_order_payment_date['order_purchase_timestamp'] = pd.to_datetime(merged_order_payment_date['order_purchase_timestamp'],
                                                                      format = '%Y-%m-%d %H:%M:%S',
                                                                      errors = 'raise')

merged_order_payment_date = merged_order_payment_date.set_index('order_purchase_timestamp')

merged_order_month_count = merged_order_payment_date.groupby(pd.Grouper(freq='M')).count()

In [46]:
# 월별 거래 건수 확인
merged_order_month_count.head()

,payment_value
order_purchase_timestamp,
2016-10-31,287
2016-11-30,0
2016-12-31,1
2017-01-31,797
2017-02-28,1737


In [47]:
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected = True)

In [48]:
merged_order_month_count.iplot( kind = 'bar', theme = 'white')

### (2) 일별

In [49]:
merged_order_day_count = merged_order_payment_date.groupby(pd.Grouper(freq = 'D')).count()

In [50]:
# 일별 거래 건수 확인
merged_order_day_count.head()

,payment_value
order_purchase_timestamp,
2016-10-03,8
2016-10-04,61
2016-10-05,38
2016-10-06,41
2016-10-07,40


In [51]:
# 거래 건수가 일별 단위로 나눠 볼 경우 bar 그래프는 시각적으로 보기 불편하다
# 따라서, 거래 건수가 확연히 많아지는 일별의 경우 scatter(즉, line) 그래프로 시각화하는게 보기 좋다.
merged_order_day_count.iplot( kind = 'scatter', theme = 'white')

### (3) 시간대별

In [52]:
merged_order_payment_date = merged_order[['order_purchase_timestamp', 'payment_value']].copy()
merged_order_payment_date.head()

,order_purchase_timestamp,payment_value
0,2017-10-02 10:56:33,18.12
1,2017-10-02 10:56:33,2.00
2,2017-10-02 10:56:33,18.59
3,2018-07-24 20:41:37,141.46
4,2018-08-08 08:38:49,179.12


In [53]:
merged_order_payment_date.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100739 entries, 0 to 100738
Data columns (total 2 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   order_purchase_timestamp  100739 non-null  object 
 1   payment_value             100739 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.3+ MB


In [54]:
merged_order_payment_date['order_purchase_timestamp'] = pd.to_datetime(merged_order_payment_date['order_purchase_timestamp'],
                                                                      format = '%Y-%m-%d %H:%M:%S',
                                                                       errors = 'raise')

merged_order_payment_date.info()        

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100739 entries, 0 to 100738
Data columns (total 2 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   order_purchase_timestamp  100739 non-null  datetime64[ns]
 1   payment_value             100739 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 2.3 MB


In [55]:
dir(merged_order_payment_date['order_purchase_timestamp'].dt)
# merged_order_payment_date['order_purchase_timestamp'].dt의 속성중에 주목해야 할 것은
# 하단의 day,second 등의 메소드!

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_accessors',
 '_add_delegate_accessors',
 '_constructor',
 '_delegate_method',
 '_delegate_property_get',
 '_delegate_property_set',
 '_deprecations',
 '_dir_additions',
 '_dir_deletions',
 '_freeze',
 '_get_values',
 '_reset_cache',
 'ceil',
 'date',
 'day',
 'day_name',
 'dayofweek',
 'dayofyear',
 'days_in_month',
 'daysinmonth',
 'floor',
 'freq',
 'hour',
 'is_leap_year',
 'is_month_end',
 'is_month_start',
 'is_quarter_end',
 'is_quarter_start',
 'is_year_end',
 'is_year_start',
 'microsecond',
 'minute',
 'month',
 'month_name',
 'nanosecond',
 'normalize',
 'quarter',
 'round',
 'second',
 'st

In [56]:
# dt.quarter 메소드는 날짜를 분기별로 분류하는데 사용된다.
merged_order_payment_date['order_purchase_timestamp'].dt.quarter

0         4
1         4
2         4
3         3
4         3
         ..
100734    1
100735    1
100736    3
100737    1
100738    1
Name: order_purchase_timestamp, Length: 100739, dtype: int64

### [ 사전 설정 ]
#### datetime 필드는 dt.시간 별로 필요한 부분만 추출 가능

In [57]:
# 기존의 컬럼에서 날짜, 일, 주, 달, 분기, 분 등의 컬럼을 날짜 데이터로부터 추출하여 추가한다.

# 몇년?
merged_order_payment_date['year'] = merged_order_payment_date['order_purchase_timestamp'].dt.year
# 몇일?
merged_order_payment_date['day'] = merged_order_payment_date['order_purchase_timestamp'].dt.day
# 몇째주?
merged_order_payment_date['weekday'] = merged_order_payment_date['order_purchase_timestamp'].dt.weekday
# 몇월?
merged_order_payment_date['month'] = merged_order_payment_date['order_purchase_timestamp'].dt.month
# 몇시?
merged_order_payment_date['hour'] = merged_order_payment_date['order_purchase_timestamp'].dt.hour
# 몇분기?
merged_order_payment_date['quarter'] = merged_order_payment_date['order_purchase_timestamp'].dt.quarter
# 몇분?
merged_order_payment_date['minute'] = merged_order_payment_date['order_purchase_timestamp'].dt.minute

In [58]:
merged_order_payment_date.head()

,order_purchase_timestamp,payment_value,year,day,weekday,month,hour,quarter,minute
0,2017-10-02 10:56:33,18.12,2017,2,0,10,10,4,56
1,2017-10-02 10:56:33,2.00,2017,2,0,10,10,4,56
2,2017-10-02 10:56:33,18.59,2017,2,0,10,10,4,56
3,2018-07-24 20:41:37,141.46,2018,24,1,7,20,3,41
4,2018-08-08 08:38:49,179.12,2018,8,2,8,8,3,38


### (4) 연도별

In [59]:
merged_order_payment_year = merged_order_payment_date[['year','payment_value']].copy()
merged_order_payment_year.head()

,year,payment_value
0,2017,18.12
1,2017,2.00
2,2017,18.59
3,2018,141.46
4,2018,179.12


In [60]:
merged_order_payment_year = merged_order_payment_year.groupby('year').sum()
merged_order_payment_year.head()

,payment_value
year,
2016,47290.82
2017,6920422.83
2018,8451969.20


In [61]:
# 2015, 2016.6, 2017.5와 같은 인덱스가 걸리적거린다.
# 아래 아래 코드의 layout 옵션을 통해서 thick 제거
merged_order_payment_year.iplot(kind = 'bar', theme = 'white')

#### 은근히 x tick 제어가 필요할 때가 꽤 있음

In [62]:
layout = {
    'xaxis' : {
        # (상기) showticklabels는 index를 표출할거냐 안할거냐와 관련된 옵션이다.
        'showticklabels' : True,
        'tickvals' : [2016, 2017, 2018]
    }
}

merged_order_payment_year.iplot(kind = 'bar', theme = 'white', layout = layout)

### (5) 요일별

In [63]:
merged_order_payment_weekday = merged_order_payment_date[['weekday','payment_value']].copy()

In [64]:
merged_order_payment_weekday = merged_order_payment_weekday.groupby('weekday').sum()
merged_order_payment_weekday.head()

,payment_value
weekday,
0,2530671.56
1,2473930.77
2,2396215.34
3,2283849.51
4,2222421.36


In [65]:
merged_order_payment_weekday = merged_order_payment_weekday.reset_index()
merged_order_payment_weekday.head()

,weekday,payment_value
0,0,2530671.56
1,1,2473930.77
2,2,2396215.34
3,3,2283849.51
4,4,2222421.36


In [66]:
def func(row):
    if row['weekday'] == 0:
        row['weekday'] = '일요일'
    elif row['weekday'] == 1:
        row['weekday'] = '월요일'
    elif row['weekday'] == 2:
        row['weekday'] = '화요일'
    elif row['weekday'] == 3:
        row['weekday'] = '수요일'
    elif row['weekday'] == 4:
        row['weekday'] = '목요일'
    elif row['weekday'] == 5:
        row['weekday'] = '금요일'
    elif row['weekday'] == 6:
        row['weekday'] = '토요일'
        
    return row

In [67]:
merged_order_payment_weekday = merged_order_payment_weekday.apply(func, axis = 1)
merged_order_payment_weekday.head()

,weekday,payment_value
0,일요일,2530671.56
1,월요일,2473930.77
2,화요일,2396215.34
3,수요일,2283849.51
4,목요일,2222421.36


In [68]:
merged_order_payment_weekday = merged_order_payment_weekday.set_index('weekday')
merged_order_payment_weekday.head()

,payment_value
weekday,
일요일,2530671.56
월요일,2473930.77
화요일,2396215.34
수요일,2283849.51
목요일,2222421.36


In [69]:
merged_order_payment_weekday.iplot(kind = 'bar', theme = 'white')

### (6) 계절별(quarter별) 분석
- quarter : (1:1분기, 2:2분기, 3:3분기, 4:4분기) 

> 물론 현지 사정에 따라 계절 상황을 알면 도움이 됨

In [70]:
merged_order_payment_quarter = merged_order_payment_date[['quarter', 'payment_value']].copy()
merged_order_payment_quarter = merged_order_payment_quarter.groupby('quarter').sum()
merged_order_payment_quarter.head()

,payment_value
quarter,
1,3977098.94
2,4721382.50
3,3926177.93
4,2795023.48


In [71]:
merged_order_payment_quarter = merged_order_payment_quarter.reset_index()

In [72]:
def func(row):
    if row['quarter'] == 1:
        row['quarter'] = '1Q'
    elif row['quarter'] == 2:
        row['quarter'] = '2Q'
    elif row['quarter'] == 3:
        row['quarter'] = '3Q'
    elif row['quarter'] == 4:
        row['quarter'] = '4Q'

    return row

In [73]:
merged_order_payment_quarter = merged_order_payment_quarter.apply(func, axis = 1)
merged_order_payment_quarter = merged_order_payment_quarter.set_index('quarter')
merged_order_payment_quarter.head()

,payment_value
quarter,
1Q,3977098.94
2Q,4721382.50
3Q,3926177.93
4Q,2795023.48


In [74]:
merged_order_payment_quarter.iplot(kind = 'bar', theme = 'white')

### (7) 시간대별

> 분명히 시간대별 구매율이 높은 구간이 있을 것임

In [75]:
merged_order_payment_hour = merged_order_payment_date[['hour', 'payment_value']].copy()
merged_order_payment_hour = merged_order_payment_hour.groupby('hour').sum()
merged_order_payment_hour.head()

,payment_value
hour,
0,360158.45
1,171441.04
2,64572.21
3,37551.29
4,28209.97


In [76]:
# peak 타임이 언제인지 유추가능
merged_order_payment_hour.iplot(kind = 'bar', theme ='white')

#### x tick을 시간간격으로 빼곡히 넣고 싶을 경우...

In [77]:
layout = dict(
    xaxis = {
        'showticklabels' : True,
        'dtick' : 1
    }
)

In [78]:
merged_order_payment_hour.iplot(kind = 'bar', theme ='white', layout = layout)

### (8) 분별 분석

In [79]:
merged_order_payment_minute = merged_order_payment_date[['minute', 'payment_value']].copy()
merged_order_payment_minute = merged_order_payment_minute.groupby('minute').sum()
merged_order_payment_minute.head()

,payment_value
minute,
0,253410.99
1,247286.12
2,220971.41
3,245626.28
4,265087.98


In [80]:
# 별 도움은 안되는듯 분다위는...
merged_order_payment_minute.iplot(kind='bar', theme='white')

### (9) 요일/시간 간 거래액 상관관계
- 상관관계 분석을 위해 사용되는 그래프 : heatmap

In [92]:
merged_order_payment_date.head()

,order_purchase_timestamp,payment_value,year,day,weekday,month,hour,quarter,minute
0,2017-10-02 10:56:33,18.12,2017,2,0,10,10,4,56
1,2017-10-02 10:56:33,2.00,2017,2,0,10,10,4,56
2,2017-10-02 10:56:33,18.59,2017,2,0,10,10,4,56
3,2018-07-24 20:41:37,141.46,2018,24,1,7,20,3,41
4,2018-08-08 08:38:49,179.12,2018,8,2,8,8,3,38


In [93]:
merged_order_payment_hour_weekday = merged_order_payment_date[['payment_value','weekday','hour']].copy()
merged_order_payment_hour_weekday.head()

,payment_value,weekday,hour
0,18.12,0,10
1,2.00,0,10
2,18.59,0,10
3,141.46,1,20
4,179.12,2,8


In [95]:
# groupby를 이용해서 weekday와 hour을 일관되게 묶어준다.
# 1주에 0~23시간 2주에 0~23시간 이렇게 묶이도록
merged_order_payment_hour_weekday = merged_order_payment_hour_weekday.groupby(['weekday', 'hour']).sum()
merged_order_payment_hour_weekday

payment_value
weekday hour               
0       0          46807.42
        1          17734.77
        2           7462.56
        3           6442.62
        4           2138.02
...                     ...
6       19        140127.39
        20        150543.96
        21        135431.31
        22        127866.74
        23         82991.95

[168 rows x 1 columns]

In [128]:
# 위의 데이터프레임의 인덱스는 2차배열로 대략[0][0]~[6*24+23][1]의 형태로 이루어졌으므로, 다음과같이 일부분을 출력할 수 있다.
print(merged_order_payment_hour_weekday.index[1])
print(merged_order_payment_hour_weekday.index[1][1])
print(merged_order_payment_hour_weekday.index[2])
print(merged_order_payment_hour_weekday.index[6*24+23][1])

(0, 1)
1
(0, 2)
23


In [87]:
# 숫자를 요일로 바꾸기 위한 apply용 함수 생성
def func(row):
    
    if row['weekday'] == 0:
        row['weekday'] = '일요일'
    elif row['weekday'] == 1:
        row['weekday'] = '월요일'
    elif row['weekday'] == 2:
        row['weekday'] = '화요일'
    elif row['weekday'] == 3:
        row['weekday'] = '수요일'
    elif row['weekday'] == 4:
        row['weekday'] = '목요일'
    elif row['weekday'] == 5:
        row['weekday'] = '금요일'
    elif row['weekday'] == 6:
        row['weekday'] = '토요일'
        
    return row

In [130]:
merged_order_payment_hour_weekday = merged_order_payment_hour_weekday.reset_index()
merged_order_payment_hour_weekday.head()

,weekday,hour,payment_value
0,0,0,46807.42
1,0,1,17734.77
2,0,2,7462.56
3,0,3,6442.62
4,0,4,2138.02


In [134]:
merged_order_payment_hour_weekday = merged_order_payment_hour_weekday.apply(func, axis = 1)
merged_order_payment_hour_weekday

,weekday,hour,payment_value
0,일요일,0.0,46807.42
1,일요일,1.0,17734.77
2,일요일,2.0,7462.56
3,일요일,3.0,6442.62
4,일요일,4.0,2138.02
...,...,...,...
163,토요일,19.0,140127.39
164,토요일,20.0,150543.96
165,토요일,21.0,135431.31
166,토요일,22.0,127866.74


### 행 임의 이름 순서로 정렬하기
- 우선 해당 이름에 번호를 매겨 놓은 후, sort_values 로 번호를 기반으로 값으로 정렬함

```python
해당데이터프레임Series = pd.Categorical(시리즈데이터, categories=이름순서리스트, ordered=True)
```

In [135]:
merged_order_payment_hour_weekday['weekday'] = pd.Categorical(merged_order_payment_hour_weekday['weekday'],
                                                             # sort하기위해 요일명에 순서를 부여함
                                                             categories = ['월요일','화요일','수요일','목요일','금요일','토요일','일요일'],
                                                             ordered = True)

In [138]:
merged_order_payment_hour_weekday = merged_order_payment_hour_weekday.sort_values(by = ['weekday','hour'])
# weekday와 함께 hour도 정렬시켜줘야 시간의 순서가 뒤죽박죽 섞이지 않는다.
merged_order_payment_hour_weekday

,weekday,hour,payment_value
24,월요일,0.0,46897.54
25,월요일,1.0,23357.21
26,월요일,2.0,9775.06
27,월요일,3.0,5766.02
28,월요일,4.0,4011.98
...,...,...,...
19,일요일,19.0,155189.14
20,일요일,20.0,162640.98
21,일요일,21.0,174686.78
22,일요일,22.0,165825.81


In [139]:
merged_order_payment_hour_weekday.iplot(kind = 'heatmap',
                                       y = 'weekday',
                                       x = 'hour',
                                       z = 'payment_value',
                                       colorscale = 'Blues')

In [142]:
layout = {
    'title' : {
        'text' : "<b>Heatmap of turnover over the hour by day</b>",
        'x' : 0.5,
        'y' : 1,
        'font' : {
            'size' : 16
        }
    },
    
    'xaxis' : {
        'title' : '<b>Hour</b>',
        'showticklabels' : True,
        'dtick' : 1
    },
    
    'yaxis' : {
        'title' : '<b>The day of the week</b>'
    },
    
    "template":'plotly_white'
}

merged_order_payment_hour_weekday.iplot(kind = 'heatmap',
                                       y = 'weekday',
                                       x = 'hour',
                                       z = 'payment_value',
                                       colorscale = 'Reds',
                                       layout = layout)